In [1]:
%pwd


'C:\\Users\\godar\\Desktop\\Text-Summarize-Project\\research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'C:\\Users\\godar\\Desktop\\Text-Summarize-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path
# Entity(which defines the return type of function
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL : str
    local_data_file:Path
    unzip_dir:Path

In [6]:
!pip install python-box

In [7]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config =read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(root_dir=config.root_dir,
                                                  source_URL=config.source_URL,
                                                  local_data_file=config.local_data_file,
                                                  unzip_dir=config.unzip_dir)
        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size


In [10]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url = self.config.source_URL,
                                                    filename = self.config.local_data_file)
            logger.info(f"Download file {filename} with following headers: {headers}")
        else:
            logger.info(f"File {self.config.local_data_file} already exists of size:{get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zf:
            zf.extractall(unzip_path)

In [11]:
try:
   config=ConfigurationManager()
   data_ingestion_config=config.get_data_ingestion_config()
   data_ingestion=DataIngestion(config=data_ingestion_config)
   data_ingestion.download_file()
   data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-27 15:46:47,991] : INFO : common : Loaded config\config.yaml successfully
[2025-03-27 15:46:47,994] : INFO : common : Loaded params.yaml successfully
[2025-03-27 15:46:47,995] : INFO : common : Created directory artifacts successfully
[2025-03-27 15:46:47,996] : INFO : common : Created directory artifacts/data_ingestion successfully
[2025-03-27 15:46:50,193] : INFO : 2666824989 : Download file artifacts/data_ingestion/data.zip with following headers: Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2612:2942F9:150F05:2EB5F4:67E5258F
Accept-Ranges: bytes
Date: Thu, 27 Mar 2025 10:16:48 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10